In [1]:
import os

In [2]:
%pwd

'f:\\kidney-Disease-Classification_DL\\research'

In [3]:
os.chdir("f:/kidney-Disease-Classification_DL")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataInjectionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=Config_PATH,
                 params_filepath=Params_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_injection_config(self) -> DataInjectionConfig:
        config = self.config.data_injection
        create_directories([config.root_dir])
        data_injection_config = DataInjectionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_injection_config

In [12]:
import gdown
import zipfile
import os
from pathlib import Path

from CNNClassifier.utils.common import get_size
from CNNClassifier import logger


class DataInjection:
    def __init__(self, config: DataInjectionConfig):
        self.config = config

    def download_file(self):
        """
        Download file from Google Drive using gdown
        """
        os.makedirs("artifacts/data_injection", exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            logger.info(
                f"Downloading file from: [{self.config.source_URL}] "
                f"to: [{self.config.local_data_file}]"
            )
            gdown.download(
                url=self.config.source_URL,
                output=str(self.config.local_data_file),
                quiet=False
            )
        else:
            logger.info(
                f"File already exists of size: "
                f"[{get_size(Path(self.config.local_data_file))}]"
            )

    def extract_zip_file(self):
        """
        Extracts the downloaded zip file into the configured unzip directory
        """
        unzip_dir = Path(self.config.unzip_dir)
        os.makedirs(unzip_dir, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
            logger.info(f"Extracted zip file to: {unzip_dir}")


In [13]:
try:
    config = ConfigurationManager()
    data_injection_config = config.get_data_injection_config()
    print(data_injection_config)
    data_ingestion = DataInjection(config=data_injection_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-28 13:01:49,362: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-28 13:01:49,365: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-28 13:01:49,368: INFO: common]: Created directory at: artifacts
[2025-09-28 13:01:49,370: INFO: common]: Created directory at: artifacts/data_injection
DataInjectionConfig(root_dir=WindowsPath('artifacts/data_injection'), source_URL='https://drive.google.com/uc?id=1japQFug4eo3bz9mKibbk-aR4YJVhU4H1', local_data_file=WindowsPath('artifacts/data_injection/data.zip'), unzip_dir=WindowsPath('artifacts/data_injection'))
[2025-09-28 13:01:49,373: INFO: 3970809810]: Downloading file from: [https://drive.google.com/uc?id=1japQFug4eo3bz9mKibbk-aR4YJVhU4H1] to: [artifacts\data_injection\data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1japQFug4eo3bz9mKibbk-aR4YJVhU4H1
From (redirected): https://drive.google.com/uc?id=1japQFug4eo3bz9mKibbk-aR4YJVhU4H1&confirm=t&uuid=5f291fd1-e8c9-4be5-85ab-f2617fd4ba76
To: f:\kidney-Disease-Classification_DL\artifacts\data_injection\data.zip
100%|██████████| 1.63G/1.63G [05:30<00:00, 4.92MB/s]


[2025-09-28 13:08:27,482: INFO: 3970809810]: Extracted zip file to: artifacts\data_injection
